In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler
from xgboost.sklearn import XGBRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor

import time
import os
import random
from collections import Counter

pd.set_option('display.max_columns', None)

df = pd.read_csv('TEST/대곡/2012 - 복사본.csv', encoding='cp949')

df.drop(columns='강수량', inplace=True)
df

,일시,50CM 정시 토양수분(%),0.5M 정시 습도(%),1.5M 정시 습도(%),4.0M 정시 습도(%),10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),0.5M 정시 기온(°C),1.5M 정시 기온(°C),4.0M 정시 기온(°C),1.5M 정시 풍속(m/s),4.0M 정시 풍속(m/s),정시 지면온도(°C),5CM 정시 지중온도(°C),10CM 정시 지중온도(°C),20CM 정시 지중온도(°C),30CM 정시 지중온도(°C),정시 누적 순복사(MJ/m2),정시 누적 전천복사(MJ/m2),정시 누적 반사복사(MJ/m2)
0,2012-01-01 0:00,10.0,71.1,70.8,66.6,16.9,11.7,6.4,-3.1,-2.9,-2.6,0.3,0.3,-3.1,0.0,0.5,1.7,2.6,7.12,10.37,3.25
1,2012-01-01 1:00,10.0,76.0,76.2,71.3,17.0,11.7,6.4,-4.2,-3.8,-3.5,0.3,0.5,-3.9,0.0,0.5,1.6,2.6,0.00,0.00,0.00
2,2012-01-01 2:00,10.0,78.5,78.4,74.2,17.0,11.7,6.4,-4.9,-4.6,-4.2,0.4,0.3,-4.4,-0.1,0.4,1.6,2.6,0.00,0.00,0.00
3,2012-01-01 3:00,10.0,81.8,81.8,78.8,17.0,11.7,6.4,-5.5,-5.3,-5.1,0.0,0.0,-4.7,-0.2,0.3,1.6,2.6,0.00,0.00,0.00
4,2012-01-01 4:00,10.0,83.3,83.3,84.4,16.9,11.7,6.4,-5.8,-5.5,-5.2,0.1,0.0,-4.9,-0.3,0.3,1.6,2.6,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455875,2017-12-31 19:00,29.2,58.0,51.2,53.3,5.9,25.8,25.6,2.6,2.7,2.7,2.2,3.2,2.5,4.1,4.4,4.6,4.6,4.16,6.65,2.49
455876,2017-12-31 20:00,29.2,65.3,58.1,58.8,5.9,25.8,25.6,0.4,0.6,0.6,0.4,0.6,1.7,3.7,4.1,4.5,4.6,4.16,6.65,2.49
455877,2017-12-31 21:00,29.2,78.0,65.8,65.2,5.9,25.8,25.6,-1.1,-0.6,-0.3,0.4,0.6,1.3,3.3,3.9,4.4,4.6,4.16,6.65,2.49
455878,2017-12-31 22:00,29.2,82.5,73.0,74.5,5.9,25.8,25.6,-1.7,-1.5,-1.5,0.2,0.1,1.0,2.9,3.6,4.2,4.6,4.16,6.65,2.49


In [2]:
df_col = df.columns[1:]
df_col

Index(['50CM 정시 토양수분(%)', '0.5M 정시 습도(%)', '1.5M 정시 습도(%)', '4.0M 정시 습도(%)',
       '10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',
       '0.5M 정시 기온(°C)', '1.5M 정시 기온(°C)', '4.0M 정시 기온(°C)', '1.5M 정시 풍속(m/s)',
       '4.0M 정시 풍속(m/s)', '정시 지면온도(°C)', '5CM 정시 지중온도(°C)', '10CM 정시 지중온도(°C)',
       '20CM 정시 지중온도(°C)', '30CM 정시 지중온도(°C)', '정시 누적 순복사(MJ/m2)',
       '정시 누적 전천복사(MJ/m2)', '정시 누적 반사복사(MJ/m2)'],
      dtype='object')

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df.iloc[:, 1:])
df_scaled = scaler.transform(df.iloc[:, 1:])

x, y = df_scaled[:,1:], df_scaled[:,0]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=111)
print("x_train Data shape: {}".format(x_train.shape))
print("x_test Data shape: {}".format(x_test.shape))
print("y_train Data shape: {}".format(y_train.shape))
print("y_test Data shape: {}".format(y_test.shape))

x_train Data shape: (341910, 19)
x_test Data shape: (113970, 19)
y_train Data shape: (341910,)
y_test Data shape: (113970,)


## 랜덤포레스트모델

In [96]:
#시간 체크용
start = time.time()     

rf= RandomForestRegressor(n_jobs=-1, random_state=111)

param_grid = {
    'n_estimators':[50, 100, 150, 200, 400],
    'max_depth':list(range(5, 20)),
    'min_samples_leaf':list(range(2, 5)),
    'min_samples_split':list(range(1, 5)),
    'max_samples': [0.2, 0.4, 0.7, 0.8, 0.9, 1.0],
    'max_features': [0.2, 0.4, 0.7, 0.8, 0.9, 1.0]
    }

# ss=ShuffleSplit(test_size=0.3,random_state=111)
grid_rf_cv=RandomizedSearchCV(rf, 
                              param_distributions=param_grid, 
                              cv=5, 
                             verbose=1,
                             n_jobs=-1, 
                             random_state=111)

grid_rf_cv.fit(x_train,y_train)

end=time.time()
print('수행 시간: {0:.3f}'.format(end - start))
print('최적의 매개변수 조합: ', grid_rf_cv.best_params_)
print('최고의 교차 검증 점수: ', grid_rf_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\ensemble\_forest.py", line 476, in fit
    trees = Parallel(
  File "C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\joblib\parallel.py", line 1056, in __call__
    self.retrieve()
  File "C:\Users\leedo\Miniconda3\e

수행 시간: 529.541
최적의 매개변수 조합:  {'n_estimators': 100, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_samples': 0.9, 'max_features': 0.8, 'max_depth': 19}
최고의 교차 검증 점수:  0.9746649101605046


In [97]:
rf = RandomForestRegressor(n_estimators=grid_rf_cv.best_params_['n_estimators'] , 
                          max_depth=grid_rf_cv.best_params_['max_depth'] ,
                          min_samples_leaf=grid_rf_cv.best_params_['min_samples_leaf'] ,
                          min_samples_split=grid_rf_cv.best_params_['min_samples_split'] ,
                           max_samples=grid_rf_cv.best_params_['max_samples'] ,
                           max_features=grid_rf_cv.best_params_['max_features'] ,
                          n_jobs = -1, 
                           random_state = 111)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

In [9]:
mae = mean_absolute_error(y_pred, y_test)
mse = mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_pred, y_test)
print('MAE 평균절대오차 낮을수록좋다 : ', round(mae, 3))
print('MSE 평균제곱근오차 낮을수록좋다 : ', round(mse, 3))
print('RMSE 낮을수록좋다:', round(rmse, 3))
print('r2 score 1에 가까울수록 좋다:', round(r2, 3))

MAE 평균절대오차 낮을수록좋다 :  0.056
MSE 평균제곱근오차 낮을수록좋다 :  0.021
RMSE 낮을수록좋다: 0.145
r2 score 1에 가까울수록 좋다: 0.977


## XGB모델

In [21]:
#시간 체크용
start = time.time()     

reg = XGBRegressor(n_jobs=-1, random_state=111)

param_grid = {
    'n_estimators':[50, 100, 150, 200, 400],
    'max_depth':list(range(5, 20)),
    'gamma': [0.01, 0.02, 0.03],
    'importance_type': ['weight', 'gain', 'cover', 'total_gain', 'total_cover'],
    'reg_lambda': [0.1, 0.2, 0.3, 0.4, 0.7, 0.8, 0.9, 1.0]
    }

# ss=ShuffleSplit(test_size=0.3,random_state=111)
grid_rf_cv=RandomizedSearchCV(reg, 
                              param_distributions=param_grid, 
                              cv=5, 
                             verbose=1,
                             n_jobs=-1, 
                             random_state=111)

grid_rf_cv.fit(x_train,y_train)

end=time.time()
print('수행 시간: {0:.3f}'.format(end - start))
print('최적의 매개변수 조합: ', grid_rf_cv.best_params_)
print('최고의 교차 검증 점수: ', grid_rf_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
수행 시간: 2112.405
최적의 매개변수 조합:  {'reg_lambda': 0.9, 'n_estimators': 150, 'max_depth': 19, 'importance_type': 'gain', 'gamma': 0.02}
최고의 교차 검증 점수:  0.9782496616591617


In [22]:
reg = XGBRegressor(n_estimators=grid_rf_cv.best_params_['n_estimators'] , 
                  max_depth=grid_rf_cv.best_params_['max_depth'] ,
                  gamma=grid_rf_cv.best_params_['gamma'] ,
                  importance_type=grid_rf_cv.best_params_['importance_type'] ,
                   reg_lambda=grid_rf_cv.best_params_['reg_lambda'] ,
                  n_jobs = -1, 
                   random_state = 111)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

In [105]:
reg = XGBRegressor(
            n_estimators=100, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
            max_depth=20, ## 개별 나무의 최대 깊이
            gamma = 0.03, ## gamma
            importance_type='weight', ## gain, weight, cover, total_gain, total_cover
            reg_lambda = 0.3, ## tuning parameter of l2 penalty
            random_state=111
        )
reg.fit(x_train,y_train)
y_pred =reg.predict(x_test)

In [23]:
mae = mean_absolute_error(y_pred, y_test)
mse = mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_pred, y_test)
print('MAE 평균절대오차 낮을수록좋다 : ', round(mae, 3))
print('MSE 평균제곱근오차 낮을수록좋다 : ', round(mse, 3))
print('RMSE 낮을수록좋다:', round(rmse, 3))
print('r2 score 1에 가까울수록 좋다:', round(r2, 3))

MAE 평균절대오차 낮을수록좋다 :  0.041
MSE 평균제곱근오차 낮을수록좋다 :  0.016
RMSE 낮을수록좋다: 0.128
r2 score 1에 가까울수록 좋다: 0.983


## SVR모델 실패사유: 너무 오래걸림

In [14]:
sv_regressor = SVR(kernel='rbf',
                   degree=4,
                   gamma='scale',
                   coef0=0.0,
                   tol=0.001,
                   C=1.0,
                   epsilon=0.1,
                   shrinking=True,
                   cache_size=200,
                   verbose=False,
                   max_iter=-1)

In [ ]:
sv_regressor.fit(x_train,y_train)
y_pred =sv_regressor.predict(x_test)

In [11]:
mae = mean_absolute_error(y_pred, y_test)
mse = mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_pred, y_test)
print('MAE 평균절대오차 낮을수록좋다 : ', round(mae, 3))
print('MSE 평균제곱근오차 낮을수록좋다 : ', round(mse, 3))
print('RMSE 낮을수록좋다:', round(rmse, 3))
print('r2 score 1에 가까울수록 좋다:', round(r2, 3))

MAE 평균절대오차 낮을수록좋다 :  0.279
MSE 평균제곱근오차 낮을수록좋다 :  0.186
RMSE 낮을수록좋다: 0.432
r2 score 1에 가까울수록 좋다: 0.768


In [ ]:
#시간 체크용
start = time.time()     

sv_regressor = SVR()

param_grid = {
    'coef0':list(range(1, 100)),
    'degree':list(range(3, 10)),
    'gamma': ['scale', 'auto'],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
    'C': [0.5, 0.7, 0.8, 1.0, 1.2, 1.3, 1.5, 1.7]
    }

# ss=ShuffleSplit(test_size=0.3,random_state=111)
grid_rf_cv=RandomizedSearchCV(sv_regressor, 
                              param_distributions=param_grid, 
                              cv=5, 
                             verbose=1,
                             n_jobs=-1, 
                             random_state=111)

grid_rf_cv.fit(x_train,y_train)

end=time.time()
print('수행 시간: {0:.3f}'.format(end - start))
print('최적의 매개변수 조합: ', grid_rf_cv.best_params_)
print('최고의 교차 검증 점수: ', grid_rf_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


## LGBM모델

In [144]:
lgbm = LGBMRegressor(
    n_estimators=100, 
    num_leaves = 100,
    max_depth=15, 
    min_split_gain = 0.01,
    min_child_weight = 0.001,
    min_child_samples = 20,
    boosting_type= 'gbdt',
    reg_lambda = 0.01, 
    learning_rate = 0.5,
    random_state=111
        )

In [145]:
lgbm.fit(x_train,y_train)
y_pred =lgbm.predict(x_test)

In [146]:
mae = mean_absolute_error(y_pred, y_test)
mse = mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_pred, y_test)
print('MAE 평균절대오차 낮을수록좋다 : ', round(mae, 3))
print('MSE 평균제곱근오차 낮을수록좋다 : ', round(mse, 3))
print('RMSE 낮을수록좋다:', round(rmse, 3))
print('r2 score 1에 가까울수록 좋다:', round(r2, 3))

MAE 평균절대오차 낮을수록좋다 :  0.125
MSE 평균제곱근오차 낮을수록좋다 :  0.043
RMSE 낮을수록좋다: 0.208
r2 score 1에 가까울수록 좋다: 0.955


In [7]:
#시간 체크용
start = time.time()     

lgbm = LGBMRegressor(n_jobs=-1, random_state=111)

param_grid = {
    'n_estimators':[50, 100, 150, 200, 400],
    'num_leaves':list(range(30, 500, 10)),
    'min_split_gain': [0.0, 0.01, 0.02, 0.03, 0.04, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.2, 0.3, 0.4],
    'min_child_weight':[0.0, 0.001, 0.002, 0.003, 0.004, 0.01, 0.02, 0.03, 0.04, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.2, 0.3, 0.4],
    'min_child_samples':list(range(20, 50)),
    'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5],
    'importance_type': ['gbdt', 'dart', 'goss', 'rf'],
    'reg_lambda': [0.1, 0.2, 0.3, 0.4, 0.7, 0.8, 0.9, 1.0]
    }

# ss=ShuffleSplit(test_size=0.3,random_state=111)
grid_rf_cv=RandomizedSearchCV(lgbm, 
                              param_distributions=param_grid, 
                              cv=5, 
                             verbose=1,
                             n_jobs=-1, 
                             random_state=111)

grid_rf_cv.fit(x_train,y_train)

end=time.time()
print('수행 시간: {0:.3f}'.format(end - start))
print('최적의 매개변수 조합: ', grid_rf_cv.best_params_)
print('최고의 교차 검증 점수: ', grid_rf_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
수행 시간: 55.362
최적의 매개변수 조합:  {'reg_lambda': 0.7, 'num_leaves': 400, 'n_estimators': 400, 'min_split_gain': 0.01, 'min_child_weight': 0.1, 'min_child_samples': 39, 'learning_rate': 0.1, 'importance_type': 'rf'}
최고의 교차 검증 점수:  0.9736223663643567


In [8]:
lgbm = LGBMRegressor(n_estimators=grid_rf_cv.best_params_['n_estimators'] , 
                  num_leaves=grid_rf_cv.best_params_['num_leaves'] ,
                  min_split_gain=grid_rf_cv.best_params_['min_split_gain'] ,
                   min_child_weight=grid_rf_cv.best_params_['min_child_weight'] ,
                     min_child_samples=grid_rf_cv.best_params_['min_child_samples'] ,
                     learning_rate=grid_rf_cv.best_params_['learning_rate'] ,
                     importance_type=grid_rf_cv.best_params_['importance_type'] ,
                     reg_lambda=grid_rf_cv.best_params_['reg_lambda'] ,
                  n_jobs = -1, 
                   random_state = 111)

lgbm.fit(x_train, y_train)
y_pred = lgbm.predict(x_test)

In [9]:
mae = mean_absolute_error(y_pred, y_test)
mse = mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_pred, y_test)
print('MAE 평균절대오차 낮을수록좋다 : ', round(mae, 3))
print('MSE 평균제곱근오차 낮을수록좋다 : ', round(mse, 3))
print('RMSE 낮을수록좋다:', round(rmse, 3))
print('r2 score 1에 가까울수록 좋다:', round(r2, 3))

MAE 평균절대오차 낮을수록좋다 :  0.071
MSE 평균제곱근오차 낮을수록좋다 :  0.022
RMSE 낮을수록좋다: 0.148
r2 score 1에 가까울수록 좋다: 0.977


## 미학습된 데이터에서 오차범위검증

In [177]:
check_list = []

In [168]:
def val(num):
    for i in range(num):
        randint = random.randint(0, len(a))
        if True in a.loc[randint].isnull()[2:].values:
            pass
        else:
            temp = a.loc[randint].values[2:]
            new=[]
            for i in temp:
                new.append(i)

            new_p = new.pop(6)
            new.insert(0, new_p)
            new = [new]
            new_scaled = scaler.transform(new)
            new_x , new_y = new_scaled[:, 1:], new_scaled[:, 0]
            new_pred = lgbm.predict(new_x)
            new_scaled[:,0] = new_pred
            pred = scaler.inverse_transform(new_scaled)
            check_list.append(round(abs(new[0][0] - pred[0][0]), 1))
#             print(new[0][0], '***', round(pred[0][0], 1), '***', round(abs(new[0][0] - pred[0][0]), 1))

In [179]:
a = pd.read_csv('TEST/보성/SURFACE_AGR_981_HR_2012_2012_2016.csv', encoding='cp949')
val(1000)

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

In [181]:
a = pd.read_csv('TEST/보성/SURFACE_AGR_981_HR_2013_2013_2016.csv', encoding='cp949')
val(1000)

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

In [182]:
a = pd.read_csv('TEST/보성/SURFACE_AGR_981_HR_2015_2015_2018.csv', encoding='cp949')
val(1000)

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

In [183]:
a = pd.read_csv('TEST/보성/SURFACE_AGR_981_HR_2016_2016_2018.csv', encoding='cp949')
val(1000)

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

In [184]:
a = pd.read_csv('TEST/오창가곡/SURFACE_AGR_977_HR_2018_2018_2019.csv', encoding='cp949')
val(1000)

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

In [185]:
a = pd.read_csv('TEST/춘천신북/SURFACE_AGR_978_HR_2012_2012_2016.csv', encoding='cp949')
val(1000)

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

In [186]:
a = pd.read_csv('TEST/화순능주/SURFACE_AGR_973_HR_2018_2018_2019.csv', encoding='cp949')
val(1000)

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\leedo\Miniconda3\envs\ldh\lib\site-packag

In [187]:
# 오차범위 평균, 무작위 검사 시행개수
np.mean(check_list), len(check_list)

(1.1586143695014661, 5456)

In [188]:
# 오차범위 최소, 최대
min(check_list), max(check_list)

(0.0, 23.7)

In [189]:
over_mean_list=[]
for i in check_list:
    if i > np.mean(check_list):
        over_mean_list.append(i)

In [190]:
# 오차범위 평균값 초과
len(over_mean_list)

1708

In [191]:
# 평균을 초과하는 값들의 최소, 최대
min(over_mean_list), max(over_mean_list)

(1.2, 23.7)

In [192]:
# 오차범위의 구간별 개수
Counter(sorted(over_mean_list))

Counter({1.2: 146,
         1.3: 128,
         1.4: 88,
         1.5: 121,
         1.6: 83,
         1.7: 90,
         1.8: 66,
         1.9: 56,
         2.0: 61,
         2.1: 52,
         2.2: 58,
         2.3: 68,
         2.4: 48,
         2.5: 35,
         2.6: 57,
         2.7: 39,
         2.8: 24,
         2.9: 41,
         3.0: 41,
         3.1: 13,
         3.2: 21,
         3.3: 25,
         3.4: 22,
         3.5: 24,
         3.6: 24,
         3.7: 14,
         3.8: 17,
         3.9: 15,
         4.0: 9,
         4.1: 11,
         4.2: 14,
         4.3: 12,
         4.4: 4,
         4.5: 11,
         4.6: 4,
         4.7: 12,
         4.8: 2,
         4.9: 4,
         5.0: 4,
         5.1: 7,
         5.2: 2,
         5.3: 4,
         5.4: 5,
         5.5: 8,
         5.6: 7,
         5.7: 8,
         5.8: 3,
         5.9: 3,
         6.0: 1,
         6.1: 1,
         6.2: 2,
         6.3: 2,
         6.4: 5,
         6.5: 4,
         6.6: 2,
         6.7: 5,
         6.8